# Obtain Labels

These labels have the information that I know I want to work with and will allow me to match the label to the record in the database that I need to pull.

In [1]:
import pandas as pd
to_skip_info = []
for i in range(34,1406):
    to_skip_info.append(i)
information_about_dna_data = pd.read_csv('files/GCF_000001405.39_GRCh38.p13_assembly_report_fixed.csv', skiprows=(to_skip_info), header=0)
pd.DataFrame(information_about_dna_data['Assembly name:  GRCh38.p13'].dropna(axis=0))

,Assembly name: GRCh38.p13
1,Description: Genome Reference Consortium Hu...
3,Organism name: Homo sapiens (human)
5,Taxid: 9606
7,BioProject: PRJNA31257
9,Submitter: Genome Reference Consortium
11,Date: 2019-02-28
13,Assembly type: haploid-with-alt-loci
15,Release type: patch
17,Assembly level: Chromosome
19,Genome representation: full


In [2]:
dna_data = pd.read_csv('files/GCF_000001405.39_GRCh38.p13_assembly_report_fixed.csv', header=124)
dna_data = dna_data.dropna(axis=0)
dna_data.sample(3)

,# Sequence-Name,Sequence-Role,Assigned-Molecule,Assigned-Molecule-Location/Type,GenBank-Accn,Relationship,RefSeq-Accn,Assembly-Unit,Sequence-Length,UCSC-style-name
605,HG2288_HG2289_PATCH,fix-patch,13,Chromosome,KN538371.1,=,NW_011332698.1,PATCHES,206320.0,na
1121,HSCHR22_1_CTG6,alt-scaffold,22,Chromosome,KI270878.1,=,NT_187632.1,ALT_REF_LOCI_1,186262.0,chr22_KI270878v1_alt
401,HSCHR1_3_CTG3,novel-patch,1,Chromosome,KQ458382.1,=,NW_014040925.1,PATCHES,141019.0,na


In [4]:
protein_data = pd.read_csv('files/ProteinTable51_582967.csv')
protein_data.sample(3)

,#Replicon Name,Replicon Accession,Start,Stop,Strand,GeneID,Locus,Protein product,Length,Protein name
63075,10,NC_000010.11,24380965,24546324,+,56243,KIAA1217,XP_016871911.1,1758,sickle tail protein homolog isoform X11
16730,1,NC_000001.11,223541836,223665646,-,388743,CAPN8,NP_001137434.1,703,calpain-8
107629,19,NC_000019.10,51367005,51368577,-,125875,CLDND2,XP_016881735.1,396,claudin domain-containing protein 2 isoform X2


# Explore Database

Since, I've never worked with this database, I need to pull a record to determine if the information I need is present, namely the protein sequence.

In [25]:
# prepare to contact database
import json
def get_keys(path):
    with open(path) as f:
        return json.load(f)
keys = get_keys("/Users/eannefawcett/.secret/ncbi_api.json")
api_key = keys['api_key']

## NCBI Database Information

This call to the database will tell me what type of information is housed in what containers, and allow me to select a specific database to call.

In [26]:
# pull down general database information
from Bio import Entrez
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
handle = Entrez.einfo()
result = handle.read()
handle.close()

In [27]:
print(result)

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20190110//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20190110/einfo.dtd">
<eInfoResult>
<DbList>

	<DbName>pubmed</DbName>
	<DbName>protein</DbName>
	<DbName>nuccore</DbName>
	<DbName>ipg</DbName>
	<DbName>nucleotide</DbName>
	<DbName>structure</DbName>
	<DbName>sparcle</DbName>
	<DbName>genome</DbName>
	<DbName>annotinfo</DbName>
	<DbName>assembly</DbName>
	<DbName>bioproject</DbName>
	<DbName>biosample</DbName>
	<DbName>blastdbinfo</DbName>
	<DbName>books</DbName>
	<DbName>cdd</DbName>
	<DbName>clinvar</DbName>
	<DbName>gap</DbName>
	<DbName>gapplus</DbName>
	<DbName>grasp</DbName>
	<DbName>dbvar</DbName>
	<DbName>gene</DbName>
	<DbName>gds</DbName>
	<DbName>geoprofiles</DbName>
	<DbName>homologene</DbName>
	<DbName>medgen</DbName>
	<DbName>mesh</DbName>
	<DbName>ncbisearch</DbName>
	<DbName>nlmcatalog</DbName>
	<DbName>omim</DbName>
	<DbName>orgtrack</DbName>
	<DbName>pmc</DbName>
	<DbName>p

In [28]:
handle = Entrez.einfo()
record = Entrez.read(handle)

In [29]:
record.keys()

dict_keys(['DbList'])

In [30]:
record['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']

## Protein Database Information

In [31]:
# selected database information
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
handle = Entrez.einfo(db="protein")
record = Entrez.read(handle)
record["DbInfo"]["Description"]

'Protein sequence record'

In [32]:
record["DbInfo"]["Count"]

'831488909'

In [33]:
record["DbInfo"]["LastUpdate"]

'2020/03/16 21:46'

In [34]:
record["DbInfo"].keys()

dict_keys(['DbName', 'MenuName', 'Description', 'DbBuild', 'Count', 'LastUpdate', 'FieldList', 'LinkList'])

In [35]:
for field in record["DbInfo"]["FieldList"]:
    print("%(Name)s, %(FullName)s, %(Description)s" % field)

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to each sequence
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
TITL, Title, Words in definition line
KYWD, Keyword, Nonstandardized terms provided by submitter
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
VOL, Volume, Volume number of publication
ISS, Issue, Issue number of publication
PAGE, Page Number, Page number(s) of publication
ORGN, Organism, Scientific and common names of organism, and all higher levels of taxonomy
ACCN, Accession, Accession number of sequence
PACC, Primary Accession, Does not include retired secondary accessions
GENE, Gene Name, Name of gene associated with sequence
PROT, Protein Name, Name of protein associated with sequence
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
PDAT, Publication Date, Date sequence added to GenBank
MDAT, Modification Date, Date of last update
SUBS, S

In [36]:
# look at one protein record
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
handle = Entrez.einfo(db="protein", ACCN='NP_001164209.1')
record = Entrez.read(handle)

In [37]:
record

{'DbInfo': {'DbName': 'protein', 'MenuName': 'Protein', 'Description': 'Protein sequence record', 'DbBuild': 'Build200315-0206m.1', 'Count': '831488909', 'LastUpdate': '2020/03/16 21:46', 'FieldList': [{'Name': 'ALL', 'FullName': 'All Fields', 'Description': 'All terms from all searchable fields', 'TermCount': '4427725646', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hierarchy': 'N', 'IsHidden': 'N'}, {'Name': 'UID', 'FullName': 'UID', 'Description': 'Unique number assigned to each sequence', 'TermCount': '0', 'IsDate': 'N', 'IsNumerical': 'Y', 'SingleToken': 'Y', 'Hierarchy': 'N', 'IsHidden': 'Y'}, {'Name': 'FILT', 'FullName': 'Filter', 'Description': 'Limits the records', 'TermCount': '260', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'Y', 'Hierarchy': 'N', 'IsHidden': 'N'}, {'Name': 'WORD', 'FullName': 'Text Word', 'Description': 'Free text associated with record', 'TermCount': '1159081284', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hierarchy': 'N', '

In [38]:
record.keys()

dict_keys(['DbInfo'])

In [39]:
record['DbInfo'].keys()

dict_keys(['DbName', 'MenuName', 'Description', 'DbBuild', 'Count', 'LastUpdate', 'FieldList', 'LinkList'])

In [40]:
record['DbInfo']['DbName']

'protein'

In [41]:
record['DbInfo']['MenuName']

'Protein'

In [42]:
record['DbInfo']['Description']

'Protein sequence record'

In [43]:
record['DbInfo']['DbBuild']

'Build200315-0206m.1'

In [44]:
record['DbInfo']['Count']

'831488909'

In [45]:
record['DbInfo']['LastUpdate']

'2020/03/16 21:46'

In [46]:
len(record['DbInfo']['FieldList'])

33

In [47]:
for field in record["DbInfo"]["FieldList"]:
    print("%(Name)s, %(FullName)s, %(Description)s" % field)

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to each sequence
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
TITL, Title, Words in definition line
KYWD, Keyword, Nonstandardized terms provided by submitter
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
VOL, Volume, Volume number of publication
ISS, Issue, Issue number of publication
PAGE, Page Number, Page number(s) of publication
ORGN, Organism, Scientific and common names of organism, and all higher levels of taxonomy
ACCN, Accession, Accession number of sequence
PACC, Primary Accession, Does not include retired secondary accessions
GENE, Gene Name, Name of gene associated with sequence
PROT, Protein Name, Name of protein associated with sequence
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
PDAT, Publication Date, Date sequence added to GenBank
MDAT, Modification Date, Date of last update
SUBS, S

In [48]:
for field in record['DbInfo']['FieldList']:
    print(field)
    print('\n')

{'Name': 'ALL', 'FullName': 'All Fields', 'Description': 'All terms from all searchable fields', 'TermCount': '4427725646', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hierarchy': 'N', 'IsHidden': 'N'}


{'Name': 'UID', 'FullName': 'UID', 'Description': 'Unique number assigned to each sequence', 'TermCount': '0', 'IsDate': 'N', 'IsNumerical': 'Y', 'SingleToken': 'Y', 'Hierarchy': 'N', 'IsHidden': 'Y'}


{'Name': 'FILT', 'FullName': 'Filter', 'Description': 'Limits the records', 'TermCount': '260', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'Y', 'Hierarchy': 'N', 'IsHidden': 'N'}


{'Name': 'WORD', 'FullName': 'Text Word', 'Description': 'Free text associated with record', 'TermCount': '1159081284', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hierarchy': 'N', 'IsHidden': 'N'}


{'Name': 'TITL', 'FullName': 'Title', 'Description': 'Words in definition line', 'TermCount': '134102182', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hierarchy': 'N', '

In [49]:
for field in record["DbInfo"]["LinkList"]:
    print("%(Name)s, %(Menu)s, %(Description)s" % field)

nuccore_protein_wp, Links to autonomous proteins, Autonomous protein records
protein_biocollections, BioCollections, BioCollections
protein_bioproject, BioProject Links, Proteins related to BioProjects
protein_biosystems, BioSystem Links, BioSystems
protein_ccds, ccds, Link to Consensus CDS
protein_cdd, Conserved Domain Links, Link to conserved domains within a protein
protein_cdd_concise_2, Concise Conserved Domain Links, Link to a concise list of conserved domains in a protein
protein_cdd_summary, , Display conserved domains within a protein
protein_gene, Gene Links, Link to related Genes
protein_genome, Genome Links, Genome record encoding protein sequence
protein_homologene, HomoloGene Links, Related HomoloGene
protein_nuccore, Nucleotide Links, Nucleotide sequence from coding region
protein_nuccore_mat_peptide, Mature Peptides, Link between nucleotide record and mature peptides contained within nucleotide record
protein_nuccore_mgc, Nucleotide NIH cDNA clone links, Nucleotide NIH 

In [50]:
for field in record['DbInfo']['LinkList']:
    print(field)
    print('\n')

{'Name': 'nuccore_protein_wp', 'Menu': 'Links to autonomous proteins', 'Description': 'Autonomous protein records', 'DbTo': 'nuccore'}


{'Name': 'protein_biocollections', 'Menu': 'BioCollections', 'Description': 'BioCollections', 'DbTo': 'biocollections'}


{'Name': 'protein_bioproject', 'Menu': 'BioProject Links', 'Description': 'Proteins related to BioProjects', 'DbTo': 'bioproject'}


{'Name': 'protein_biosystems', 'Menu': 'BioSystem Links', 'Description': 'BioSystems', 'DbTo': 'biosystems'}


{'Name': 'protein_ccds', 'Menu': 'ccds', 'Description': 'Link to Consensus CDS', 'DbTo': 'ccds'}


{'Name': 'protein_cdd', 'Menu': 'Conserved Domain Links', 'Description': 'Link to conserved domains within a protein', 'DbTo': 'cdd'}


{'Name': 'protein_cdd_concise_2', 'Menu': 'Concise Conserved Domain Links', 'Description': 'Link to a concise list of conserved domains in a protein', 'DbTo': 'cdd'}


{'Name': 'protein_cdd_summary', 'Menu': '', 'Description': 'Display conserved domains within a

In [51]:
# look at two data base records
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = ['NP_001164209.1', 'XP_016874800.1']
print(Entrez.epost("protein", id=",".join(id_list)).read())

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE ePostResult PUBLIC "-//NLM//DTD epost 20090526//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20090526/epost.dtd"><ePostResult>
	<QueryKey>1</QueryKey>
	<WebEnv>NCID_1_190607792_130.14.18.97_9001_1584415437_1741210555_0MetA0_S_MegaStore</WebEnv>
</ePostResult>



In [52]:
# store two database records
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = ['NP_001164209.1', 'XP_016874800.1']
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [53]:
# look at stored database records
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]
print(webenv)
print(query_key)

NCID_1_190536794_130.14.22.76_9001_1584415437_1110160424_0MetA0_S_MegaStore
1


## Specific Retrieval Mode = XML

### For 1 record

In [54]:
# change retrival mode
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = ['NP_001164209.1']
search_results = Entrez.read(Entrez.efetch(db="protein", id=id_list, retmode='xml'))

In [55]:
search_results[0].keys()

dict_keys(['GBSeq_locus', 'GBSeq_length', 'GBSeq_moltype', 'GBSeq_topology', 'GBSeq_division', 'GBSeq_update-date', 'GBSeq_create-date', 'GBSeq_definition', 'GBSeq_primary-accession', 'GBSeq_accession-version', 'GBSeq_other-seqids', 'GBSeq_keywords', 'GBSeq_source', 'GBSeq_organism', 'GBSeq_taxonomy', 'GBSeq_references', 'GBSeq_comment', 'GBSeq_source-db', 'GBSeq_feature-table', 'GBSeq_sequence'])

In [56]:
search_results[0]['GBSeq_locus']

'NP_001164209'

In [57]:
search_results[0]['GBSeq_length']

'1182'

In [58]:
search_results[0]['GBSeq_moltype']

'AA'

In [59]:
search_results[0]['GBSeq_topology']

'linear'

In [60]:
search_results[0]['GBSeq_division']

'PRI'

In [61]:
search_results[0]['GBSeq_update-date']

'25-OCT-2019'

In [62]:
search_results[0]['GBSeq_create-date']

'31-DEC-2009'

In [63]:
search_results[0]['GBSeq_definition']

'IQ motif and SEC7 domain-containing protein 3 isoform 1 [Homo sapiens]'

In [64]:
search_results[0]['GBSeq_primary-accession']

'NP_001164209'

In [65]:
search_results[0]['GBSeq_accession-version']

'NP_001164209.1'

In [66]:
search_results[0]['GBSeq_other-seqids'][0]

'ref|NP_001164209.1|'

In [67]:
search_results[0]['GBSeq_other-seqids'][1]

'gi|282721018'

In [68]:
search_results[0]['GBSeq_keywords']

['RefSeq', 'MANE Select']

In [69]:
search_results[0]['GBSeq_source']

'Homo sapiens (human)'

In [70]:
search_results[0]['GBSeq_organism']

'Homo sapiens'

In [71]:
search_results[0]['GBSeq_taxonomy']

'Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Hominidae; Homo'

In [72]:
search_results[0]['GBSeq_references'][0]

{'GBReference_reference': '1', 'GBReference_position': '1..1182', 'GBReference_authors': ['Um JW', 'Choii G', 'Park D', 'Kim D', 'Jeon S', 'Kang H', 'Mori T', 'Papadopoulos T', 'Yoo T', 'Lee Y', 'Kim E', 'Tabuchi K', 'Ko J'], 'GBReference_title': 'IQ Motif and SEC7 Domain-containing Protein 3 (IQSEC3) Interacts with Gephyrin to Promote Inhibitory Synapse Formation', 'GBReference_journal': 'J. Biol. Chem. 291 (19), 10119-10130 (2016)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1074/jbc.M115.712893'}], 'GBReference_pubmed': '27002143', 'GBReference_remark': 'GeneRIF: these data reveal that IQSEC3 acts together with gephyrin to regulate inhibitory synapse development.'}

In [73]:
search_results[0]['GBSeq_references'][1]

{'GBReference_reference': '2', 'GBReference_position': '1..1182', 'GBReference_authors': ['Turner ST', 'Bailey KR', 'Schwartz GL', 'Chapman AB', 'Chai HS', 'Boerwinkle E'], 'GBReference_title': 'Genomic association analysis identifies multiple loci influencing antihypertensive response to an angiotensin II receptor blocker', 'GBReference_journal': 'Hypertension 59 (6), 1204-1211 (2012)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1161/HYP.0b013e31825b30f8'}], 'GBReference_pubmed': '22566498'}

In [74]:
search_results[0]['GBSeq_references'][2]

{'GBReference_reference': '3', 'GBReference_position': '1..1182', 'GBReference_authors': ['Hattori Y', 'Ohta S', 'Hamada K', 'Yamada-Okabe H', 'Kanemura Y', 'Matsuzaki Y', 'Okano H', 'Kawakami Y', 'Toda M'], 'GBReference_title': 'Identification of a neuron-specific human gene, KIAA1110, that is a guanine nucleotide exchange factor for ARF1', 'GBReference_journal': 'Biochem. Biophys. Res. Commun. 364 (4), 737-742 (2007)', 'GBReference_xref': [{'GBXref_dbname': 'doi', 'GBXref_id': '10.1016/j.bbrc.2007.10.041'}], 'GBReference_pubmed': '17981261', 'GBReference_remark': 'GeneRIF: These results suggest that KIAA1110 is expressed specifically in mature neurons and may play an important role in the secretion pathway as a GEF for ARF1.'}

In [75]:
search_results[0]['GBSeq_comment']

'VALIDATED REFSEQ: This record has undergone validation or preliminary review. The reference sequence was derived from AC026369.21 and AC007406.1.; ~Transcript Variant: This variant (1) represents the longer transcript and encodes the longer isoform (1).; ~Sequence Note: The RefSeq transcript and protein were derived from genomic sequence to make the sequence consistent with the reference genome assembly. The genomic coordinates used for the transcript record were based on transcript alignments and orthologous data.; ; ##Evidence-Data-START## ; RNAseq introns :: single sample supports all introns SAMEA2145743, SAMEA2154665 [ECO:0000348] ; ##Evidence-Data-END##; ; ##RefSeq-Attributes-START## ; MANE Ensembl match :: ENST00000538872.6/ ENSP00000437554.1 ; RefSeq Select criteria :: based on conservation, expression, longest protein ; ##RefSeq-Attributes-END##'

In [76]:
search_results[0]['GBSeq_source-db']

'REFSEQ: accession NM_001170738.2'

In [77]:
len(search_results[0]['GBSeq_feature-table'])

9

In [78]:
for feature in search_results[0]['GBSeq_feature-table']:
    print(feature)
    print('\n')

{'GBFeature_key': 'source', 'GBFeature_location': '1..1182', 'GBFeature_intervals': [{'GBInterval_from': '1', 'GBInterval_to': '1182', 'GBInterval_accession': 'NP_001164209.1'}], 'GBFeature_quals': [{'GBQualifier_name': 'organism', 'GBQualifier_value': 'Homo sapiens'}, {'GBQualifier_name': 'db_xref', 'GBQualifier_value': 'taxon:9606'}, {'GBQualifier_name': 'chromosome', 'GBQualifier_value': '12'}, {'GBQualifier_name': 'map', 'GBQualifier_value': '12p13.33'}]}


{'GBFeature_key': 'Protein', 'GBFeature_location': '1..1182', 'GBFeature_intervals': [{'GBInterval_from': '1', 'GBInterval_to': '1182', 'GBInterval_accession': 'NP_001164209.1'}], 'GBFeature_quals': [{'GBQualifier_name': 'product', 'GBQualifier_value': 'IQ motif and SEC7 domain-containing protein 3 isoform 1'}, {'GBQualifier_name': 'note', 'GBQualifier_value': 'IQ motif and SEC7 domain-containing protein 3; IQ motif and Sec7 domain 3'}, {'GBQualifier_name': 'calculated_mol_wt', 'GBQualifier_value': '127491'}]}


{'GBFeature_key'

In [79]:
search_results[0]['GBSeq_feature-table'][0].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_quals'])

In [80]:
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_intervals'][0])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_intervals'][0]['GBInterval_from'])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])

source
1..1182
{'GBInterval_from': '1', 'GBInterval_to': '1182', 'GBInterval_accession': 'NP_001164209.1'}
1
1182
{'GBQualifier_name': 'organism', 'GBQualifier_value': 'Homo sapiens'}
{'GBQualifier_name': 'db_xref', 'GBQualifier_value': 'taxon:9606'}
{'GBQualifier_name': 'chromosome', 'GBQualifier_value': '12'}
12


In [81]:
search_results[0]['GBSeq_feature-table'][1].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_quals'])

In [82]:
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_intervals'][0])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_intervals'][0]['GBInterval_from'])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])

Protein
1..1182
{'GBInterval_from': '1', 'GBInterval_to': '1182', 'GBInterval_accession': 'NP_001164209.1'}
1
1182
{'GBQualifier_name': 'product', 'GBQualifier_value': 'IQ motif and SEC7 domain-containing protein 3 isoform 1'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'IQ motif and SEC7 domain-containing protein 3; IQ motif and Sec7 domain 3'}
{'GBQualifier_name': 'calculated_mol_wt', 'GBQualifier_value': '127491'}
127491


In [83]:
search_results[0]['GBSeq_feature-table'][2].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_partial5', 'GBFeature_quals'])

In [84]:
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_intervals'][0])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_intervals'][0]['GBInterval_from'])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_partial5'])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][2]['GBFeature_quals'][2]['GBQualifier_value'])

Region
<32..63
{'GBInterval_from': '32', 'GBInterval_to': '63', 'GBInterval_accession': 'NP_001164209.1'}
32
63

{'GBQualifier_name': 'region_name', 'GBQualifier_value': 'bZIP'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'Basic leucine zipper (bZIP) domain of bZIP transcription factors: a DNA-binding and dimerization domain; cd14686'}
{'GBQualifier_name': 'db_xref', 'GBQualifier_value': 'CDD:269834'}
CDD:269834


In [85]:
search_results[0]['GBSeq_feature-table'][2]['GBFeature_partial5']

StringElement('', attributes={'value': 'true'})

In [86]:
search_results[0]['GBSeq_feature-table'][3].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_operator', 'GBFeature_quals'])

In [87]:
print(search_results[0]['GBSeq_feature-table'][3]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][3]['GBFeature_location'])
for i in range(0, len(search_results[0]['GBSeq_feature-table'][3]['GBFeature_intervals'])):
    print(search_results[0]['GBSeq_feature-table'][3]['GBFeature_intervals'][i])
print(search_results[0]['GBSeq_feature-table'][3]['GBFeature_operator'])
for i in search_results[0]['GBSeq_feature-table'][3]['GBFeature_quals']:
    print(i)

Site
order(32,35..36,39..40,42..43,46..47,49..50,53..54,56..57,60..61,63)
{'GBInterval_point': '32', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '35', 'GBInterval_to': '36', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '39', 'GBInterval_to': '40', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '42', 'GBInterval_to': '43', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '46', 'GBInterval_to': '47', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '49', 'GBInterval_to': '50', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '53', 'GBInterval_to': '54', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '56', 'GBInterval_to': '57', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '60', 'GBInterval_to': '61', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_point': '63', 'GBInterval_accession': 'NP_001164209.1'}
order
{'GBQualifier_name': 'site_type', 'GBQualifier_value':

In [88]:
search_results[0]['GBSeq_feature-table'][4].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_quals'])

In [89]:
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_intervals'][0])
# print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_intervals'][0]['GBInterval_from'])
# print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][4]['GBFeature_quals'][2]['GBQualifier_value'])

Site
259
{'GBInterval_point': '259', 'GBInterval_accession': 'NP_001164209.1'}
{'GBQualifier_name': 'site_type', 'GBQualifier_value': 'phosphorylation'}
{'GBQualifier_name': 'experiment', 'GBQualifier_value': 'experimental evidence, no additional details recorded'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'Phosphoserine. {ECO:0000250|UniProtKB:Q3TES0}; propagated from UniProtKB/Swiss-Prot (Q9UPP2.3)'}
Phosphoserine. {ECO:0000250|UniProtKB:Q3TES0}; propagated from UniProtKB/Swiss-Prot (Q9UPP2.3)


In [90]:
search_results[0]['GBSeq_feature-table'][5].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_quals'])

In [91]:
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_intervals'][0])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_intervals'][0]['GBInterval_from'])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][5]['GBFeature_quals'][2]['GBQualifier_value'])

Region
651..839
{'GBInterval_from': '651', 'GBInterval_to': '839', 'GBInterval_accession': 'NP_001164209.1'}
651
839
{'GBQualifier_name': 'region_name', 'GBQualifier_value': 'Sec7'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'Sec7 domain; pfam01369'}
{'GBQualifier_name': 'db_xref', 'GBQualifier_value': 'CDD:279680'}
CDD:279680


In [92]:
search_results[0]['GBSeq_feature-table'][6].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_operator', 'GBFeature_quals'])

In [93]:
print(search_results[0]['GBSeq_feature-table'][6]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][6]['GBFeature_location'])
for i in range(0, len(search_results[0]['GBSeq_feature-table'][6]['GBFeature_intervals'])):
    print(search_results[0]['GBSeq_feature-table'][6]['GBFeature_intervals'][i])
print(search_results[0]['GBSeq_feature-table'][6]['GBFeature_operator'])
for i in search_results[0]['GBSeq_feature-table'][6]['GBFeature_quals']:
    print(i)

Site
order(742..749,781..794)
{'GBInterval_from': '742', 'GBInterval_to': '749', 'GBInterval_accession': 'NP_001164209.1'}
{'GBInterval_from': '781', 'GBInterval_to': '794', 'GBInterval_accession': 'NP_001164209.1'}
order
{'GBQualifier_name': 'site_type', 'GBQualifier_value': 'active'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'active site/putative ARF binding site [active]'}
{'GBQualifier_name': 'db_xref', 'GBQualifier_value': 'CDD:238100'}


In [94]:
search_results[0]['GBSeq_feature-table'][7].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_quals'])

In [95]:
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_intervals'][0])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_intervals'][0]['GBInterval_from'])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][7]['GBFeature_quals'][2]['GBQualifier_value'])

Region
866..996
{'GBInterval_from': '866', 'GBInterval_to': '996', 'GBInterval_accession': 'NP_001164209.1'}
866
996
{'GBQualifier_name': 'region_name', 'GBQualifier_value': 'PH_IQSEC'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'IQ motif and SEC7 domain-containing protein family Pleckstrin homology domain; cd13318'}
{'GBQualifier_name': 'db_xref', 'GBQualifier_value': 'CDD:270128'}
CDD:270128


In [96]:
search_results[0]['GBSeq_feature-table'][8].keys()

dict_keys(['GBFeature_key', 'GBFeature_location', 'GBFeature_intervals', 'GBFeature_quals'])

In [97]:
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_key'])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_location'])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_intervals'][0])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_intervals'][0]['GBInterval_from'])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_intervals'][0]['GBInterval_to'])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_quals'][0])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_quals'][1])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_quals'][2])
print(search_results[0]['GBSeq_feature-table'][8]['GBFeature_quals'][2]['GBQualifier_value'])

CDS
1..1182
{'GBInterval_from': '1', 'GBInterval_to': '1182', 'GBInterval_accession': 'NP_001164209.1'}
1
1182
{'GBQualifier_name': 'gene', 'GBQualifier_value': 'IQSEC3'}
{'GBQualifier_name': 'coded_by', 'GBQualifier_value': 'NM_001170738.2:117..3665'}
{'GBQualifier_name': 'note', 'GBQualifier_value': 'isoform 1 is encoded by transcript variant 1'}
isoform 1 is encoded by transcript variant 1


In [98]:
search_results[0]['GBSeq_sequence']

'mesllenpvravlylkeltaivqnqqslihtqreridelerrldelsaenrslwehqqllqaqpppglvppssaplpaapatapaaaaraqeplqdqgqrsaaaphpapdrpprqhhgqlleqpqrgpgsrahtpqsphkhlgtqgavtdkekerppsccaaagallqhkspsalgkgvlsrrpenetvlhqfccpaadacsdlasqsdgsctqagggmedsvvaaaavaagrpsahapkaqaqelqeeeerpgagaaspragpqhkaspgrqqpalatalcphapaasdyelsldlknkqiemlehkygghlvsrraactiqtafrqyqlsknfekirnsllesrlprrislrkvrsptaeslaaekalmegyglvglplvrspslpptfagtlteledsfteqvqslaksiddalstwslktmcslresgayqlhqalqaaagppgleaegrapesagpgpgddaaetpglppahsgtlmmafrdvtvqianqnisvssstalsvanclgaqtvqapaepaagkaeqgetsgreapeapavgredasaedscaeaaasgaadgatapkteeeeeeeetaevgrgaeaeagdleqlsssststksaksgseasasaskdalqamilslpryhcenpascksptlstdtlrkrlyriglnlfninpdkgiqflisrgfipdtpigvahfllqrkglsrqmigeflgnskkqfnrdvldcvvdemdfssmeldealrkfqahirvqgeaqkverlieafsqrycmcnpevvqqfhnpdtifilafaiillntdmyspnikpdrkmmledfirnlrgvddgadiprelvvgiyeriqqkelksnedhvtyvtkveksivgmktvlsvphrrlvccsrlfevtdvnklqkqaahqrevflfndllvilklcpkkkssstytfcksvgllgmqfqlfeneyyshgitlvtplsgsekkqvlhfcalgsdemqkfvedlkesiaevteleqiriewelekqqgt

### For multiple records

In [101]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [102]:
len(results)

2

## Explore Database Conclusions

I need to pull out the following information:
- GBSeq_sequence
- GBSeq_references using line comprehension --> not in every entry
- GBSeq_organism
- GBSeq_accession-version
- GBSeq_definition
- GBSeq_division
- GBSeq_topology
- GBSeq_moltype
- ['GBSeq_feature-table'][0]['GBFeature_key'] == 'source', then ['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'] # for chromosome #
- ['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein', ['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'] # for molar weight
 - count 'GBSeq_feature-table[x]['GBFeature_key'] == 'Region'
 - count 'GBSeq_feature-table[x]['GBFeature_key'] == 'Site'

In [103]:
protein_additional_data = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data['Chromosome'] = chromosome
protein_additional_data['Accesion Version'] = accession_version
protein_additional_data['Organism'] = organism
protein_additional_data['Definition'] = definition
protein_additional_data['Division'] = division
protein_additional_data['Topology'] = topology
protein_additional_data['Molecule Type'] = moltype
protein_additional_data['Molecular Weight'] = molar_weight
protein_additional_data['Number of Regions'] = regions
protein_additional_data['Number of Binding Sites'] = sites
protein_additional_data['Sequence'] = sequence

In [104]:
protein_additional_data

,Chromosome,Accesion Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,12,NP_001164209.1,Homo sapiens,IQ motif and SEC7 domain-containing protein 3 ...,PRI,linear,AA,127491,3,3,mesllenpvravlylkeltaivqnqqslihtqreridelerrldel...
1,12,XP_016874800.1,Homo sapiens,IQ motif and SEC7 domain-containing protein 3 ...,PRI,linear,AA,112808,0,0,mesllenpvravlylkeltaivqnqqslihtqreridelerrldel...


# Chromosome and Protein Information

In [5]:
reference_accession_numbers_dna = dna_data['RefSeq-Accn'].unique()

In [7]:
decision_about_chromosome = pd.DataFrame()
accession_numbers = dna_data['RefSeq-Accn'].unique()
number_of_proteins = []
for accession_number in reference_accession_numbers_dna:
    number_of_proteins.append(len(protein_data[protein_data['Replicon Accession']==accession_number]))
decision_about_chromosome['Accession Number'] = accession_numbers
decision_about_chromosome['Number of Proteins'] = number_of_proteins
decision_about_chromosome[decision_about_chromosome['Number of Proteins'] >= 1000]

,Accession Number,Number of Proteins
0,NC_000001.11,11086
1,NC_000002.12,8442
2,NC_000003.12,7198
3,NC_000004.12,4622
4,NC_000005.10,4796
5,NC_000006.12,5618
6,NC_000007.14,5212
7,NC_000008.11,4090
8,NC_000009.12,4652
9,NC_000010.11,5420


In [108]:
protein_accession_numbers = list(protein_data[protein_data['Replicon Accession']=='NC_000021.9']['Protein product'])
protein_accession_numbers

['XP_006723962.1',
 'XP_011544381.1',
 'XP_011544380.1',
 'NP_001350699.1',
 'NP_001350689.1',
 'NP_001350690.1',
 'NP_001350687.1',
 'NP_001355167.1',
 'XP_011544384.3',
 'XP_016883695.1',
 'XP_011544387.2',
 'XP_024307804.1',
 'XP_011544385.1',
 'NP_001308973.1',
 'XP_016883696.1',
 'XP_011544386.1',
 'XP_006723981.1',
 'XP_011544388.1',
 'XP_006723982.1',
 'XP_016884017.1',
 'XP_016883697.1',
 'NP_001307572.1',
 'XP_016883999.1',
 'XP_016884000.1',
 'XP_024307912.1',
 'NP_001340935.1',
 'NP_001340936.1',
 'NP_001308002.1',
 'XP_016883699.1',
 'NP_001340944.1',
 'NP_001340937.1',
 'NP_001340943.1',
 'NP_001340941.1',
 'XP_016883698.1',
 'NP_001340938.1',
 'NP_001340939.1',
 'XP_024307806.1',
 'XP_024307805.1',
 'NP_001307575.1',
 'NP_001307580.1',
 'XP_016883709.1',
 'XP_016883708.1',
 'NP_001307579.1',
 'XP_024307807.1',
 'NP_001307577.1',
 'NP_001300979.1',
 'NP_001307648.1',
 'XP_024307809.1',
 'XP_016883711.1',
 'XP_024307808.1',
 'XP_016883710.1',
 'XP_011544404.1',
 'XP_0168837

# Chromosome 21

## Obtain Data Using API Calls to NCBI's Protein Database

### First Batch

In [109]:
# First batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[0:200]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [110]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [111]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [112]:
protein_additional_data_1 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_1['Chromosome'] = chromosome
protein_additional_data_1['Accession Version'] = accession_version
protein_additional_data_1['Organism'] = organism
protein_additional_data_1['Definition'] = definition
protein_additional_data_1['Division'] = division
protein_additional_data_1['Topology'] = topology
protein_additional_data_1['Molecule Type'] = moltype
protein_additional_data_1['Molecular Weight'] = molar_weight
protein_additional_data_1['Number of Regions'] = regions
protein_additional_data_1['Number of Binding Sites'] = sites
protein_additional_data_1['Sequence'] = sequence

In [113]:
protein_additional_data_1

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,XP_006723962.1,Homo sapiens,ICOS ligand isoform X1 [Homo sapiens],PRI,linear,AA,56181,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
1,21,XP_011544381.1,Homo sapiens,ICOS ligand isoform X3 [Homo sapiens],PRI,linear,AA,33350,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
2,21,XP_011544380.1,Homo sapiens,ICOS ligand isoform X2 [Homo sapiens],PRI,linear,AA,39507,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
3,21,NP_001350699.1,Homo sapiens,ICOS ligand precursor [Homo sapiens],PRI,linear,AA,31389,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
4,21,NP_001350689.1,Homo sapiens,glutamine amidotransferase-like class 1 domain...,PRI,linear,AA,24598,1,1,maavralvasrlaaasaftslspggrtpsqraalhlsvprpaarva...
...,...,...,...,...,...,...,...,...,...,...,...
195,21,XP_024307892.1,Homo sapiens,junctional adhesion molecule B isoform X1 [Hom...,PRI,linear,AA,22874,2,0,midfniriknvtrsdagkyrcevsapseqgqnleedtvtlevlvap...
196,21,NP_001003701.1,Homo sapiens,"ATP synthase-coupling factor 6, mitochondrial ...",PRI,linear,AA,13257,1,0,mhcdggismilqrlfrfssvirsavsvhlrrnigvtavafnkeldp...
197,21,NP_001003697.1,Homo sapiens,"ATP synthase-coupling factor 6, mitochondrial ...",PRI,linear,AA,8960,1,9,milqrlfrfssvirsavsvhlrrnigvtavafnkeldpiqklfvdk...
198,21,NP_001003696.1,Homo sapiens,"ATP synthase-coupling factor 6, mitochondrial ...",PRI,linear,AA,8960,1,9,milqrlfrfssvirsavsvhlrrnigvtavafnkeldpiqklfvdk...


In [114]:
import time
time.sleep(10)

### Subsequent Batches

In [115]:
# second batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[200:400]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [116]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [117]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [118]:
protein_additional_data_2 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_2['Chromosome'] = chromosome
protein_additional_data_2['Accession Version'] = accession_version
protein_additional_data_2['Organism'] = organism
protein_additional_data_2['Definition'] = definition
protein_additional_data_2['Division'] = division
protein_additional_data_2['Topology'] = topology
protein_additional_data_2['Molecule Type'] = moltype
protein_additional_data_2['Molecular Weight'] = molar_weight
protein_additional_data_2['Number of Regions'] = regions
protein_additional_data_2['Number of Binding Sites'] = sites
protein_additional_data_2['Sequence'] = sequence

In [119]:
protein_additional_data_2

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,NP_001676.2,Homo sapiens,"ATP synthase-coupling factor 6, mitochondrial ...",PRI,linear,AA,8960,1,9,milqrlfrfssvirsavsvhlrrnigvtavafnkeldpiqklfvdk...
1,21,NP_001307195.1,Homo sapiens,"ATP synthase-coupling factor 6, mitochondrial ...",PRI,linear,AA,12456,1,9,milqrlfrfssvirsavsvhlrrnigvtavafnkeldpiqklfvdk...
2,21,NP_001003703.1,Homo sapiens,"ATP synthase-coupling factor 6, mitochondrial ...",PRI,linear,AA,8960,1,9,milqrlfrfssvirsavsvhlrrnigvtavafnkeldpiqklfvdk...
3,21,XP_005260995.1,Homo sapiens,GA-binding protein alpha chain isoform X1 [Hom...,PRI,linear,AA,51164,3,0,mtkreaeelieieidgtekaecteesiveqtyapaecvsqaidine...
4,21,NP_001184226.1,Homo sapiens,GA-binding protein alpha chain [Homo sapiens],PRI,linear,AA,51164,3,1,mtkreaeelieieidgtekaecteesiveqtyapaecvsqaidine...
...,...,...,...,...,...,...,...,...,...,...,...
195,21,NP_037461.2,Homo sapiens,PAX3- and PAX7-binding protein 1 isoform 2 [Ho...,PRI,linear,AA,93040,2,10,mfrkarrvnvrkrndseeeererdeeqepppllpppgtgeeagpgg...
196,21,XP_011528106.1,Homo sapiens,PAX3- and PAX7-binding protein 1 isoform X2 [H...,PRI,linear,AA,73726,1,0,mfrkarrvnvrkrndseeeererdeeqepppllpppgtgeeagpgg...
197,21,NP_001155967.2,Homo sapiens,uncharacterized protein C21orf62 precursor [Ho...,PRI,linear,AA,22843,1,0,mappsrhclllistlgvfalncftkgqknstliftrentirncscs...
198,21,NP_062542.5,Homo sapiens,uncharacterized protein C21orf62 precursor [Ho...,PRI,linear,AA,22843,1,0,mappsrhclllistlgvfalncftkgqknstliftrentirncscs...


In [120]:
time.sleep(10)

In [121]:
# third batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[400:600]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [122]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [123]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [124]:
protein_additional_data_3 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_3['Chromosome'] = chromosome
protein_additional_data_3['Accession Version'] = accession_version
protein_additional_data_3['Organism'] = organism
protein_additional_data_3['Definition'] = definition
protein_additional_data_3['Division'] = division
protein_additional_data_3['Topology'] = topology
protein_additional_data_3['Molecule Type'] = moltype
protein_additional_data_3['Molecular Weight'] = molar_weight
protein_additional_data_3['Number of Regions'] = regions
protein_additional_data_3['Number of Binding Sites'] = sites
protein_additional_data_3['Sequence'] = sequence

In [125]:
protein_additional_data_3

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,XP_011527955.1,Homo sapiens,uncharacterized protein C21orf62 isoform X1 [H...,PRI,linear,AA,11529,0,0,mrlvgqqlppgchgndqaagppkteknskcktthstlacrlnqlss...
1,21,NP_005797.1,Homo sapiens,oligodendrocyte transcription factor 2 [Homo s...,PRI,linear,AA,32254,1,3,mdsdaslvssrpsspepddlflparskgssgsaftggtvssstpsd...
2,21,XP_005260965.1,Homo sapiens,oligodendrocyte transcription factor 2 isoform...,PRI,linear,AA,32254,1,3,mdsdaslvssrpsspepddlflparskgssgsaftggtvssstpsd...
3,21,NP_620450.2,Homo sapiens,oligodendrocyte transcription factor 1 [Homo s...,PRI,linear,AA,27774,1,3,myyavsqarvnavpgtmlrpqrpgdlqlgaslyelvgyrqppssss...
4,21,NP_000865.2,Homo sapiens,interferon alpha/beta receptor 2 isoform b [Ho...,PRI,linear,AA,37262,3,0,mllsqnafifrslnlvlmvyislvfgisydspdytdesctfkislr...
...,...,...,...,...,...,...,...,...,...,...,...
195,21,XP_006724040.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,85453,1,4,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
196,21,XP_016883773.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,84426,1,4,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
197,21,NP_001334650.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,84426,2,20,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
198,21,NP_001387.2,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,85453,2,20,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...


In [126]:
time.sleep(10)

In [127]:
# forth batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[600:800]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [128]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [129]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [130]:
protein_additional_data_4 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_4['Chromosome'] = chromosome
protein_additional_data_4['Accession Version'] = accession_version
protein_additional_data_4['Organism'] = organism
protein_additional_data_4['Definition'] = definition
protein_additional_data_4['Division'] = division
protein_additional_data_4['Topology'] = topology
protein_additional_data_4['Molecule Type'] = moltype
protein_additional_data_4['Molecular Weight'] = molar_weight
protein_additional_data_4['Number of Regions'] = regions
protein_additional_data_4['Number of Binding Sites'] = sites
protein_additional_data_4['Sequence'] = sequence

In [131]:
protein_additional_data_4

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,XP_011527785.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,85453,1,4,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
1,21,NP_569120.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,84426,2,20,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
2,21,NP_567824.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,65968,2,4,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
3,21,NP_569122.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,60199,2,16,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
4,21,XP_006724041.1,Homo sapiens,dual specificity tyrosine-phosphorylation-regu...,PRI,linear,AA,85453,1,4,mhtggetsackpssvrlapsfsfhaaglqmagqmphshqysdrrqp...
...,...,...,...,...,...,...,...,...,...,...,...
195,21,NP_004906.3,Homo sapiens,ATP-binding cassette sub-family G member 1 iso...,PRI,linear,AA,75461,3,13,maclmaafsvgtamnassysaemtepksvcvsvdevvssnmeatet...
196,21,NP_058198.2,Homo sapiens,ATP-binding cassette sub-family G member 1 iso...,PRI,linear,AA,74024,3,13,maclmaafsvgtamnassysaemtepksvcvsvdevvssnmeatet...
197,21,XP_011528109.1,Homo sapiens,ATP-binding cassette sub-family G member 1 iso...,PRI,linear,AA,70746,3,7,mrislpraperdggvsasslldtvtnassysaemtepksvcvsvde...
198,21,NP_997057.1,Homo sapiens,ATP-binding cassette sub-family G member 1 iso...,PRI,linear,AA,75250,3,7,mrislpraperdggvsasslldtvtnassysaemtepksvcvsvde...


In [132]:
time.sleep(10)

In [133]:
# fifth batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[800:1000]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [134]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [135]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [136]:
protein_additional_data_5 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_5['Chromosome'] = chromosome
protein_additional_data_5['Accession Version'] = accession_version
protein_additional_data_5['Organism'] = organism
protein_additional_data_5['Definition'] = definition
protein_additional_data_5['Division'] = division
protein_additional_data_5['Topology'] = topology
protein_additional_data_5['Molecule Type'] = moltype
protein_additional_data_5['Molecular Weight'] = molar_weight
protein_additional_data_5['Number of Regions'] = regions
protein_additional_data_5['Number of Binding Sites'] = sites
protein_additional_data_5['Sequence'] = sequence

In [137]:
protein_additional_data_5

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,NP_997511.1,Homo sapiens,ATP-binding cassette sub-family G member 1 iso...,PRI,linear,AA,71829,3,7,mtepksvcvsvdevvssnmeatetdllnghlkkvdnnlteaqrfss...
1,21,XP_024307909.1,Homo sapiens,ATP-binding cassette sub-family G member 1 iso...,PRI,linear,AA,86625,1,0,mvrrgwsvctaillarlwclvpthtflseypeaaeyphpgwvywlq...
2,21,NP_003217.3,Homo sapiens,trefoil factor 3 precursor [Homo sapiens],PRI,linear,AA,6581,1,2,mkrvlscvpeptvvmaaralcmlglvlallssssaeeyvglsanqc...
3,21,NP_005414.1,Homo sapiens,trefoil factor 2 precursor [Homo sapiens],PRI,linear,AA,11990,2,4,mgrrdaqllaallvlglcalagsekpspcqcsrlsphnrtncgfpg...
4,21,NP_003216.1,Homo sapiens,trefoil factor 1 precursor [Homo sapiens],PRI,linear,AA,6674,1,2,matmenkvicalvlvsmlalgtlaeaqtetctvaprerqncgfpgv...
...,...,...,...,...,...,...,...,...,...,...,...
195,21,XP_024307828.1,Homo sapiens,ICOS ligand isoform X1 [Homo sapiens],PRI,linear,AA,51794,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
196,21,XP_011527817.1,Homo sapiens,ICOS ligand isoform X3 [Homo sapiens],PRI,linear,AA,33350,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
197,21,NP_037501.2,Homo sapiens,DNA (cytosine-5)-methyltransferase 3-like isof...,PRI,linear,AA,43539,1,2,maaipaldpeaepsmdvilvgsselsssvspgtgrdliayevkanq...
198,21,NP_787063.1,Homo sapiens,DNA (cytosine-5)-methyltransferase 3-like isof...,PRI,linear,AA,43452,1,2,maaipaldpeaepsmdvilvgsselsssvspgtgrdliayevkanq...


In [138]:
time.sleep(10)

In [139]:
# sixth batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[1000:1200]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [140]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [141]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [142]:
protein_additional_data_6 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_6['Chromosome'] = chromosome
protein_additional_data_6['Accession Version'] = accession_version
protein_additional_data_6['Organism'] = organism
protein_additional_data_6['Definition'] = definition
protein_additional_data_6['Division'] = division
protein_additional_data_6['Topology'] = topology
protein_additional_data_6['Molecule Type'] = moltype
protein_additional_data_6['Molecular Weight'] = molar_weight
protein_additional_data_6['Number of Regions'] = regions
protein_additional_data_6['Number of Binding Sites'] = sites
protein_additional_data_6['Sequence'] = sequence

In [143]:
protein_additional_data_6

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,NP_002617.3,Homo sapiens,"ATP-dependent 6-phosphofructokinase, liver typ...",PRI,linear,AA,84888,10,5,maavdleklrasgagkaigvltsggdaqgmnaavravtrmgiyvga...
1,21,XP_005261192.1,Homo sapiens,"ATP-dependent 6-phosphofructokinase, liver typ...",PRI,linear,AA,87546,2,0,maavdleklrasgagkaigvltsggdaqgmnaavravtrmgiyvga...
2,21,NP_001002021.2,Homo sapiens,"ATP-dependent 6-phosphofructokinase, liver typ...",PRI,linear,AA,90447,1,0,mrsmcnqgrgressrgglhvqgscrglsrspqqetgfakapagtdc...
3,21,XP_016883857.1,Homo sapiens,"ATP-dependent 6-phosphofructokinase, liver typ...",PRI,linear,AA,93105,2,0,mrsmcnqgrgressrgglhvqgscrglsrspqqetgfakapagtdc...
4,21,XP_006724074.1,Homo sapiens,"ATP-dependent 6-phosphofructokinase, liver typ...",PRI,linear,AA,93105,2,0,mrsmcnqgrgressrgglhvqgscrglsrspqqetgfakapagtdc...
...,...,...,...,...,...,...,...,...,...,...,...
195,21,XP_005261245.1,Homo sapiens,speriolin-like protein isoform X1 [Homo sapiens],PRI,linear,AA,37482,2,0,maeggelmsrllsenadlkkqvrllkenqmlrrllsqscqeggghd...
196,21,NP_001138908.1,Homo sapiens,lanosterol synthase isoform 2 [Homo sapiens],PRI,linear,AA,82068,8,3,mtegtclrrrggpyktepatdlgrwrlncergrqtwtylqderagr...
197,21,NP_002331.3,Homo sapiens,lanosterol synthase isoform 1 [Homo sapiens],PRI,linear,AA,83178,9,3,mtegtclrrrggpyktepatdlgrwrlncergrqtwtylqderagr...
198,21,NP_001001438.1,Homo sapiens,lanosterol synthase isoform 1 [Homo sapiens],PRI,linear,AA,83178,9,3,mtegtclrrrggpyktepatdlgrwrlncergrqtwtylqderagr...


In [144]:
time.sleep(10)

In [145]:
# seventh batch
Entrez.api_key = api_key
Entrez.email = 'elizabethfawcett47@gmail.com'
Entrez.tool = 'locus_by_amino_acid'
id_list = protein_accession_numbers[1200:1287]
search_results = Entrez.read(Entrez.epost("protein", id=",".join(id_list)))

In [146]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

In [147]:
results = Entrez.read(Entrez.efetch(db="protein", webenv=webenv, query_key=query_key, retmode='xml'))

In [148]:
protein_additional_data_7 = pd.DataFrame()
accession_version = []
definition = []
division = []
topology = []
moltype = []
organism = []
sequence = []
chromosome = []
molar_weight = []
regions = []
sites = []
for i in results:
    region = 0
    site = 0
    accession_version.append(i['GBSeq_accession-version'])
    organism.append(i['GBSeq_organism'])
    definition.append(i['GBSeq_definition'])
    division.append(i['GBSeq_division'])
    topology.append(i['GBSeq_topology'])
    moltype.append(i['GBSeq_moltype'])
    sequence.append(i['GBSeq_sequence'])
    if i['GBSeq_feature-table'][0]['GBFeature_key'] == 'source':
        chromosome.append(i['GBSeq_feature-table'][0]['GBFeature_quals'][2]['GBQualifier_value'])
    else:
        continue
    if i['GBSeq_feature-table'][1]['GBFeature_key'] == 'Protein':
        if i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][0]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][1]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][2]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][3]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][4]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][5]['GBQualifier_value'])
        elif i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_name'] == 'calculated_mol_wt':
            molar_weight.append(i['GBSeq_feature-table'][1]['GBFeature_quals'][6]['GBQualifier_value'])
        else:
            print('add more elif statements')
    else:
        continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Region':
            region += 1
        else:
            continue
    for j in range(0, len(i['GBSeq_feature-table'])):
        if i['GBSeq_feature-table'][j]['GBFeature_key'] == 'Site':
            site += 1
        else:
            continue
    regions.append(region)
    sites.append(site)
protein_additional_data_7['Chromosome'] = chromosome
protein_additional_data_7['Accession Version'] = accession_version
protein_additional_data_7['Organism'] = organism
protein_additional_data_7['Definition'] = definition
protein_additional_data_7['Division'] = division
protein_additional_data_7['Topology'] = topology
protein_additional_data_7['Molecule Type'] = moltype
protein_additional_data_7['Molecular Weight'] = molar_weight
protein_additional_data_7['Number of Regions'] = regions
protein_additional_data_7['Number of Binding Sites'] = sites
protein_additional_data_7['Sequence'] = sequence

In [149]:
protein_additional_data_7

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,XP_005261261.1,Homo sapiens,germinal-center associated nuclear protein iso...,PRI,linear,AA,218275,5,0,mnptnpfsgqqpsafsasssnvgtlpskppfrfgqpslfgqnstls...
1,21,XP_005261260.1,Homo sapiens,germinal-center associated nuclear protein iso...,PRI,linear,AA,218275,5,0,mnptnpfsgqqpsafsasssnvgtlpskppfrfgqpslfgqnstls...
2,21,NP_003897.2,Homo sapiens,germinal-center associated nuclear protein [Ho...,PRI,linear,AA,218275,12,7,mnptnpfsgqqpsafsasssnvgtlpskppfrfgqpslfgqnstls...
3,21,XP_005261262.1,Homo sapiens,germinal-center associated nuclear protein iso...,PRI,linear,AA,218275,5,0,mnptnpfsgqqpsafsasssnvgtlpskppfrfgqpslfgqnstls...
4,21,XP_011527936.2,Homo sapiens,endoribonuclease YbeY isoform X4 [Homo sapiens],PRI,linear,AA,33333,0,0,mwrpgraelrggrkrrpwrrpsphahrlapgprgprcplaprkdlr...
...,...,...,...,...,...,...,...,...,...,...,...
81,21,NP_001273607.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,25363,3,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
82,21,NP_001526.2,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,48911,6,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
83,21,XP_006724061.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,33412,2,2,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
84,21,NP_001229795.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,31062,5,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...


In [150]:
time.sleep(10)

### Concatenation of Batches

In [151]:
to_concat = [protein_additional_data_1, protein_additional_data_2, protein_additional_data_3, protein_additional_data_4, 
           protein_additional_data_5, protein_additional_data_6, protein_additional_data_7]

In [152]:
protein_additional_data = pd.concat(to_concat)

In [153]:
protein_additional_data

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
0,21,XP_006723962.1,Homo sapiens,ICOS ligand isoform X1 [Homo sapiens],PRI,linear,AA,56181,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
1,21,XP_011544381.1,Homo sapiens,ICOS ligand isoform X3 [Homo sapiens],PRI,linear,AA,33350,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
2,21,XP_011544380.1,Homo sapiens,ICOS ligand isoform X2 [Homo sapiens],PRI,linear,AA,39507,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
3,21,NP_001350699.1,Homo sapiens,ICOS ligand precursor [Homo sapiens],PRI,linear,AA,31389,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
4,21,NP_001350689.1,Homo sapiens,glutamine amidotransferase-like class 1 domain...,PRI,linear,AA,24598,1,1,maavralvasrlaaasaftslspggrtpsqraalhlsvprpaarva...
...,...,...,...,...,...,...,...,...,...,...,...
81,21,NP_001273607.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,25363,3,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
82,21,NP_001526.2,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,48911,6,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
83,21,XP_006724061.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,33412,2,2,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
84,21,NP_001229795.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,31062,5,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...


In [154]:
# check to see if only one row for dna data
dna_data_21 = dna_data[dna_data['RefSeq-Accn']=='NC_000021.9']
dna_data_21

,# Sequence-Name,Sequence-Role,Assigned-Molecule,Assigned-Molecule-Location/Type,GenBank-Accn,Relationship,RefSeq-Accn,Assembly-Unit,Sequence-Length,UCSC-style-name
41,21,assembled-molecule,21,Chromosome,CM000683.2,=,NC_000021.9,Primary Assembly,46709983.0,chr21


In [159]:
# check to see if number of rows is equal to the number of proteins listed for dna data accession number
protein_data_21 = protein_data[protein_data['Replicon Accession']=='NC_000021.9']
protein_data_21.index=protein_data_21['Protein product']
protein_data_21

,#Replicon Name,Replicon Accession,Start,Stop,Strand,GeneID,Locus,Protein product,Length,Protein name
Protein product,,,,,,,,,,
XP_006723962.1,21,NC_000021.9,5022680,5034096,+,102723996,LOC102723996,XP_006723962.1,516,ICOS ligand isoform X1
XP_011544381.1,21,NC_000021.9,5022680,5034096,+,102723996,LOC102723996,XP_011544381.1,304,ICOS ligand isoform X3
XP_011544380.1,21,NC_000021.9,5022680,5045606,+,102723996,LOC102723996,XP_011544380.1,361,ICOS ligand isoform X2
NP_001350699.1,21,NC_000021.9,5022680,5034592,+,102723996,LOC102723996,NP_001350699.1,302,ICOS ligand precursor
NP_001350689.1,21,NC_000021.9,5117103,5128354,-,102724023,GATD3B,NP_001350689.1,237,glutamine amidotransferase-like class 1 domain...
...,...,...,...,...,...,...,...,...,...,...
NP_001273607.1,21,NC_000021.9,46636952,46664327,+,3275,PRMT2,NP_001273607.1,228,protein arginine N-methyltransferase 2 isoform 7
NP_001526.2,21,NC_000021.9,46636952,46664327,+,3275,PRMT2,NP_001526.2,433,protein arginine N-methyltransferase 2 isoform 1
XP_006724061.1,21,NC_000021.9,46636952,46664374,+,3275,PRMT2,XP_006724061.1,301,protein arginine N-methyltransferase 2 isoform X2


In [160]:
protein_additional_data_21 = protein_additional_data
protein_additional_data_21.index = protein_additional_data_21['Accession Version']
protein_additional_data_21

,Chromosome,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
Accession Version,,,,,,,,,,,
XP_006723962.1,21,XP_006723962.1,Homo sapiens,ICOS ligand isoform X1 [Homo sapiens],PRI,linear,AA,56181,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
XP_011544381.1,21,XP_011544381.1,Homo sapiens,ICOS ligand isoform X3 [Homo sapiens],PRI,linear,AA,33350,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
XP_011544380.1,21,XP_011544380.1,Homo sapiens,ICOS ligand isoform X2 [Homo sapiens],PRI,linear,AA,39507,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
NP_001350699.1,21,NP_001350699.1,Homo sapiens,ICOS ligand precursor [Homo sapiens],PRI,linear,AA,31389,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
NP_001350689.1,21,NP_001350689.1,Homo sapiens,glutamine amidotransferase-like class 1 domain...,PRI,linear,AA,24598,1,1,maavralvasrlaaasaftslspggrtpsqraalhlsvprpaarva...
...,...,...,...,...,...,...,...,...,...,...,...
NP_001273607.1,21,NP_001273607.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,25363,3,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
NP_001526.2,21,NP_001526.2,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,48911,6,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
XP_006724061.1,21,XP_006724061.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,33412,2,2,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...


In [161]:
all_protein_data_21 = protein_data_21.join(protein_additional_data_21)
all_protein_data_21

,#Replicon Name,Replicon Accession,Start,Stop,Strand,GeneID,Locus,Protein product,Length,Protein name,...,Accession Version,Organism,Definition,Division,Topology,Molecule Type,Molecular Weight,Number of Regions,Number of Binding Sites,Sequence
Protein product,,,,,,,,,,,,,,,,,,,,,
XP_006723962.1,21,NC_000021.9,5022680,5034096,+,102723996,LOC102723996,XP_006723962.1,516,ICOS ligand isoform X1,...,XP_006723962.1,Homo sapiens,ICOS ligand isoform X1 [Homo sapiens],PRI,linear,AA,56181,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
XP_011544381.1,21,NC_000021.9,5022680,5034096,+,102723996,LOC102723996,XP_011544381.1,304,ICOS ligand isoform X3,...,XP_011544381.1,Homo sapiens,ICOS ligand isoform X3 [Homo sapiens],PRI,linear,AA,33350,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
XP_011544380.1,21,NC_000021.9,5022680,5045606,+,102723996,LOC102723996,XP_011544380.1,361,ICOS ligand isoform X2,...,XP_011544380.1,Homo sapiens,ICOS ligand isoform X2 [Homo sapiens],PRI,linear,AA,39507,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
NP_001350699.1,21,NC_000021.9,5022680,5034592,+,102723996,LOC102723996,NP_001350699.1,302,ICOS ligand precursor,...,NP_001350699.1,Homo sapiens,ICOS ligand precursor [Homo sapiens],PRI,linear,AA,31389,2,0,mrlgspgllfllfsslradtqekevramvgsdvelscacpegsrfd...
NP_001350689.1,21,NC_000021.9,5117103,5128354,-,102724023,GATD3B,NP_001350689.1,237,glutamine amidotransferase-like class 1 domain...,...,NP_001350689.1,Homo sapiens,glutamine amidotransferase-like class 1 domain...,PRI,linear,AA,24598,1,1,maavralvasrlaaasaftslspggrtpsqraalhlsvprpaarva...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_001273607.1,21,NC_000021.9,46636952,46664327,+,3275,PRMT2,NP_001273607.1,228,protein arginine N-methyltransferase 2 isoform 7,...,NP_001273607.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,25363,3,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
NP_001526.2,21,NC_000021.9,46636952,46664327,+,3275,PRMT2,NP_001526.2,433,protein arginine N-methyltransferase 2 isoform 1,...,NP_001526.2,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,48911,6,4,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...
XP_006724061.1,21,NC_000021.9,46636952,46664374,+,3275,PRMT2,XP_006724061.1,301,protein arginine N-methyltransferase 2 isoform X2,...,XP_006724061.1,Homo sapiens,protein arginine N-methyltransferase 2 isoform...,PRI,linear,AA,33412,2,2,matsgdcprsesqgeepaecseagllqegvqpeefvaiadyaatde...


# Store Data for Later Use

In [166]:
import pickle
all_protein_data_21.to_csv('files/all_protein_data_21.csv')

# References

Use of entrez package for online searches using api
- http://biopython.org/DIST/docs/tutorial/Tutorial.pdf 

# Future Work

- Find a way to pull down references while contacting the database